In [2]:
import psycopg2
from creds import local_db_password
from fixture_analysis_config import real_results_sql, local_host, local_name, local_port, local_user
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
conn = psycopg2.connect(dbname=local_name, user =local_user, host=local_host, password=local_db_password, port =local_port)
cur = conn.cursor()

Home vs Away advantage

In [ ]:
# Aggregate result perspective
results_2023 = pd.read_sql(real_results_sql, conn)
print(results_2023.columns)
results_2023.describe()

In [ ]:
from statsmodels.formula.api import ols

def model_home_vs_away(field = None, source_df = None, debug = False):
    home_df = source_df[[f'home_{field}']]
    home_df =  home_df.rename(columns={f'home_{field}':field})
    home_df['g'] = 1
    home_df = home_df.dropna(axis=0, how='any')
    away_df = source_df[[f'away_{field}']]
    away_df =  away_df.rename(columns={f'away_{field}':field})
    away_df['g'] = 0
    away_df = away_df.dropna(axis=0, how='any')
    reg_df = pd.concat([home_df, away_df])
    lr = ols(f'{field} ~ g', data=reg_df).fit()
    print(f'\n---Results for {field}------\n')
    print(f'R-Suqared: {lr.rsquared}')
    print(f'Abs Effect of Home adv: {lr.params[1]}')
    print(f'Y_Intercept: {lr.params[0]}')
    print(f'Average {field} for Home: {home_df[field].mean()}')
    print(f'Average {field} for Away: {away_df[field].mean()}')
    print(f'P-value: {lr.pvalues[1]}')
    if debug == True:
        print('-------------Debug-------')
        print('--')
        print(reg_df.head(3))
        print('--')
        print(reg_df.describe())

In [ ]:
model_home_vs_away('goal', results_2023, True)

Shooting

In [ ]:
model_home_vs_away('goal', results_2023)
model_home_vs_away('shot', results_2023)
model_home_vs_away('goalie_save_rate', results_2023)

Passing

In [ ]:
model_home_vs_away('passes_attempt', results_2023)
model_home_vs_away('pass_accuracy', results_2023)

Dribbles

In [ ]:
model_home_vs_away('dribble_attempt', results_2023)
model_home_vs_away('dribble_success_rate', results_2023)

Defense

In [ ]:
model_home_vs_away('tackles', results_2023)
model_home_vs_away('shot_blocks', results_2023)
model_home_vs_away('interception_rate', results_2023)